# Assignment

Train a MNIST Classification Model.

In [1]:
# Import Libaries
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

### Main Components

 1. Dataset 
  - MNIST
 2. Model Architecture
  - Assignment Model
 3. Loss (Update model)
  - Categorical Cross Entropy
 4. Optimizer (Regularizer)
  - Adam Optimizer
 5. Metrics (Visual for User)
  - Loss, Accuracy
 6. Training Model
 
 7. Save Model
  - Model Checkpoint

In [2]:
# Let's check if we are using GPU or CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
# You can Try Various Augmentation Method to see the improvement.
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,)),
                                transforms.RandomRotation(30)
                                ])


test_transform = transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))
])


train_dataset = torchvision.datasets.MNIST(
        './data', True, 
        transform=transform, download=True)

test_dataset = torchvision.datasets.MNIST(
        './data', False, 
        transform=test_transform, download=True)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
train_loader = torch.utils.data.DataLoader(train_dataset, 32, shuffle=True,
                                           num_workers=2)

test_loader = torch.utils.data.DataLoader(test_dataset, 8, shuffle=True,
                                           num_workers=2)

In [5]:
# You can modify the model architecture Here
class SampleModel(nn.Module):
    def __init__(self):
        super(SampleModel, self).__init__()

        ### CODE HERE ###
        self.conv1 = nn.Conv2d(1, 1, 3)
        self.fc1 = nn.Linear(26*26*1, 10)
        #################

    def forward(self, x):
        ### CODE HERE ###
        x = F.relu(self.conv1(x))
        x = x.view(-1, 26*26*1)
        x = self.fc1(x)
        #################
        return x

In [6]:
model = SampleModel()
model.to(device)


# Define Loss
criterion = nn.CrossEntropyLoss()
# Define Optimizer
optimizer = optim.Adam(model.parameters())

In [7]:
EPOCH = 10

for epoch in range(EPOCH):
    loss_epoch = 0
    total_imgs = 0
    correct = 0
    for i, data in enumerate(train_loader):
        imgs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        predicts = model(imgs)

        loss = criterion(predicts, labels)
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()

        total_imgs += len(imgs)
        correct += (torch.argmax(predicts, 1)==labels).sum().item()

    acc = (correct/total_imgs)*100
    print(f"EPOCH : {epoch+1}, Acc : {acc:.2f}, Loss : {loss:.2f}")

EPOCH : 1, Acc : 91.12, Loss : 0.09
EPOCH : 2, Acc : 95.63, Loss : 0.14
EPOCH : 3, Acc : 96.34, Loss : 0.11
EPOCH : 4, Acc : 96.69, Loss : 0.22
EPOCH : 5, Acc : 97.06, Loss : 0.17
EPOCH : 6, Acc : 97.24, Loss : 0.18
EPOCH : 7, Acc : 97.41, Loss : 0.01
EPOCH : 8, Acc : 97.60, Loss : 0.03
EPOCH : 9, Acc : 97.69, Loss : 0.20
EPOCH : 10, Acc : 97.75, Loss : 0.01


In [10]:
# Test Dataset accuracy:
num_total = 0
num_correct = 0
with torch.no_grad():
    for i, data in enumerate(test_loader):
        image, label = data
        image = image.to(device)
        label = label.to(device)

        predict = model(image)
        predict_class = torch.argmax(predict, 1)
        correct = (predict_class == label)


        num_correct += correct.sum().item()
        num_total += len(image)
    
print("Number of Images that is correct : ", num_correct)
print("Total Number of Testing Images : ", num_total)
print(f"Accuracy : {(num_correct/num_total)*100} %", )

Number of Images that is correct :  9850
Total Number of Testing Images :  10000
Accuracy : 98.5


# Assignment Score Regulation

`Testing Accuracy` > 98.5 % , `100 Points`.

`Testing Accuracy` >= 97 % , `50 Points (Passing Grade)`.

You are required to submit the `model.zip` file to this [link](https://docs.google.com/forms/d/e/1FAIpQLSd0TXieY8slvk9c-AyZeO1qgtnraF08jr81LensM-dcu6LXGg/viewform) for your assignment.

In [11]:
scripted_model = torch.jit.script(model)
print(scripted_model)
scripted_model.save('model.zip')

RecursiveScriptModule(
  original_name=SampleModel
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (fc1): RecursiveScriptModule(original_name=Linear)
)
